In [52]:
#Classfication using GLove
#Getting the data
!wget https://www.cs.umb.edu/~smimarog/textmining/datasets/r8-train-all-terms.txt

--2019-03-11 07:10:12--  https://www.cs.umb.edu/~smimarog/textmining/datasets/r8-train-all-terms.txt
Resolving www.cs.umb.edu (www.cs.umb.edu)... 158.121.106.224
Connecting to www.cs.umb.edu (www.cs.umb.edu)|158.121.106.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3354424 (3.2M) [text/plain]
Saving to: ‘r8-train-all-terms.txt.4’

r8-train-all-terms. 100%[===================>]   3.20M  7.48MB/s    in 0.4s    

2019-03-11 07:10:12 (7.48 MB/s) - ‘r8-train-all-terms.txt.4’ saved [3354424/3354424]



In [0]:
import sys

import numpy as np

import pandas as pd

import matplotlib.pyplot as plt



from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier

In [0]:
train=pd.read_csv("r8-train-all-terms.txt",sep ="\t",header=None)


In [55]:
!wget https://www.cs.umb.edu/~smimarog/textmining/datasets/r8-test-all-terms.txt

--2019-03-11 07:10:13--  https://www.cs.umb.edu/~smimarog/textmining/datasets/r8-test-all-terms.txt
Resolving www.cs.umb.edu (www.cs.umb.edu)... 158.121.106.224
Connecting to www.cs.umb.edu (www.cs.umb.edu)|158.121.106.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1195261 (1.1M) [text/plain]
Saving to: ‘r8-test-all-terms.txt.4’

r8-test-all-terms.t 100%[===================>]   1.14M  4.71MB/s    in 0.2s    

2019-03-11 07:10:14 (4.71 MB/s) - ‘r8-test-all-terms.txt.4’ saved [1195261/1195261]



In [0]:
test=pd.read_csv("r8-test-all-terms.txt",sep ="\t",header=None)

In [0]:
train.columns = ['label', 'content']

test.columns = ['label', 'content']

In [0]:
class Glove_Vectorizer:

  def __init__(self):

    # load in pre-trained word vectors

    print('Loading word vectors...')

    word2vec = {}

    embedding = []

    idx2word = []

    with open('glove.6B.50d.txt') as f:

      # is just a space-separated text file in the format:

      # word vec[0] vec[1] vec[2] ...

      for line in f:

        values = line.split()

        word = values[0]

        vec = np.asarray(values[1:], dtype='float32')

        word2vec[word] = vec

        embedding.append(vec)

        idx2word.append(word)

    print('Found %s word vectors.' % len(word2vec))



    # save for later

    self.word2vec = word2vec

    self.embedding = np.array(embedding)

    self.word2idx = {v:k for k,v in enumerate(idx2word)}

    self.V, self.D = self.embedding.shape



  def fit(self, data):

    pass



  def transform(self, data):

    X = np.zeros((len(data), self.D))

    n = 0

    emptycount = 0

    for sentence in data:

      tokens = sentence.lower().split()

      vecs = []

      for word in tokens:

        if word in self.word2vec:

          vec = self.word2vec[word]

          vecs.append(vec)

      if len(vecs) > 0:

        vecs = np.array(vecs)

        X[n] = vecs.mean(axis=0)

      else:

        emptycount += 1

      n += 1

    print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))

    return X



  def fit_transform(self, data):

    self.fit(data)

    return self.transform(data)

In [59]:
vectorizer = Glove_Vectorizer()

# vectorizer = Word2VecVectorizer()

Xtrain = vectorizer.fit_transform(train.content)

Ytrain = train.label



Xtest = vectorizer.transform(test.content)

Ytest = test.label

Loading word vectors...
Found 400000 word vectors.
Numer of samples with no words found: 0 / 5485
Numer of samples with no words found: 0 / 2189


In [60]:
# create the model, train it, print scores

model = RandomForestClassifier(n_estimators=200)

model.fit(Xtrain, Ytrain)

print("train score:", model.score(Xtrain, Ytrain))

print("test score:", model.score(Xtest, Ytest))

train score: 0.9992707383773929
test score: 0.9346733668341709
